## 들어가며

geometry에 속성을 결합하는 일은 그리 어려운 일은 아니고 GIS를 다뤄보신 분들이라면 누구나 해보는 일입니다. 하지만 테이블 갯수가 여럿이면 귀찮죠. 이 작업을 Geopandas에서 덜 귀찮은 방식으로 해보겠습니다. 이 과정에 관심이 없으신 분들은 과정은 건너뛰시고 이 페이지 맨 하단에서 결과물만 얻으시면 됩니다.


In [58]:
# 우선 필요한 라이브러리들을 불러옵니다.

import pandas as pd
import geopandas as gpd
import os

### 테이블 리스트 만들기

일단 속성 테이블의 리스트를 만들겠습니다. 속성테이블들은 sgis-data/attributes 디렉토리에 있습니다.

In [59]:
census_table_dir = "../sgis-data/attribute"
tables = list()
for root, dirs, filenames in os.walk(census_table_dir):
    for f in filenames:
        if '.txt' in f:
            tables.append(os.path.join(root, f))
tables

['../sgis-data/attribute/가구/2015년기준_2015년_가구총괄.txt',
 '../sgis-data/attribute/가구/2015년기준_2015년_세대구성별가구.txt',
 '../sgis-data/attribute/인구/2015년기준_2015년_성연령별인구.txt',
 '../sgis-data/attribute/인구/2015년기준_2015년_인구총괄(노년부양비).txt',
 '../sgis-data/attribute/인구/2015년기준_2015년_인구총괄(노령화지수).txt',
 '../sgis-data/attribute/인구/2015년기준_2015년_인구총괄(유년부양비).txt',
 '../sgis-data/attribute/인구/2015년기준_2015년_인구총괄(인구밀도).txt',
 '../sgis-data/attribute/인구/2015년기준_2015년_인구총괄(총인구).txt',
 '../sgis-data/attribute/인구/2015년기준_2015년_인구총괄(평균나이).txt',
 '../sgis-data/attribute/주택/2015년기준_2015년_연건평별주택.txt',
 '../sgis-data/attribute/주택/2015년기준_2015년_주택사용기간별주택.txt',
 '../sgis-data/attribute/주택/2015년기준_2015년_주택유형별주택.txt',
 '../sgis-data/attribute/주택/2015년기준_2015년_주택총괄_총주택(거처)수.txt']

테이블 리스트가 적당해 보입니다. 이 테이블들을 모두 하나의 dataframe으로 합쳐야합니다. 그리고 나중에 shapefile과 결합해야죠. 우선 테이블이 어떻게 생겼는지 봐야겠습니다. 0번째 테이블 하나만 들여다 봅시다.

In [60]:
pd.read_csv(tables[0], sep='^').head()

,base_year,tot_oa_cd,item,value
0,2015,1101053010001,to_ga_001,205.0
1,2015,1101053010001,to_ga_002,2.3
2,2015,1101053010002,to_ga_001,210.0
3,2015,1101053010002,to_ga_002,2.3
4,2015,1101053010003,to_ga_001,211.0


### long form을 wide form으로 

변수 이름이 item항목에, 값이 value에 들어가 있는 long form으로 되어있습니다. geometry와 붙이려면 wide form으로 바꾸어야겠죠. 각 column이 변수가 되도록요. 그리고 tot_oa_cd가 각 집계구 id인 것 같으니 이것은 string으로 저장하는 것이 낫겠습니다. 

In [61]:
# 아래 함수에서 pivot이 있는 부분이 각 csv를 wide form 형태의 
# dataframe으로 바꾸고 있습니다.
def csv2df(table_path):
    df = pd.read_csv(table_path, sep='^', dtype={'tot_oa_cd':str})
    df = df.pivot(index='tot_oa_cd', columns='item', values='value')
    return df

# 각각의 테이블들을 dataframe들로 바꾸고
att_dfs = [csv2df(table) for table in tables]
# 합칩니다. 넓은 방향(axis=1)으로요.
att_df = pd.concat(dfs, axis=1)
att_df.head()

item,to_ga_001,to_ga_002,to_ga_999,ga_sd_001,ga_sd_002,ga_sd_003,ga_sd_004,ga_sd_005,ga_sd_006,ga_sd_999,...,ho_pr_999,ho_gb_001,ho_gb_002,ho_gb_003,ho_gb_004,ho_gb_005,ho_gb_006,ho_gb_999,to_ho_001,to_ho_999
tot_oa_cd,,,,,,,,,,,,,,,,,,,,,
1101053010001,205.0,2.3,NaN,37.0,92.0,7.0,NaN,63.0,5.0,NaN,...,NaN,46.0,96.0,NaN,NaN,5.0,16.0,NaN,147.0,NaN
1101053010002,210.0,2.3,NaN,29.0,86.0,14.0,NaN,76.0,NaN,NaN,...,NaN,24.0,93.0,18.0,9.0,NaN,NaN,NaN,148.0,NaN
1101053010003,211.0,2.1,NaN,35.0,80.0,10.0,NaN,78.0,8.0,NaN,...,NaN,8.0,97.0,8.0,NaN,10.0,9.0,NaN,123.0,NaN
1101053010004,181.0,2.4,NaN,24.0,72.0,15.0,NaN,66.0,NaN,NaN,...,NaN,16.0,89.0,12.0,NaN,NaN,6.0,NaN,123.0,NaN
1101053010005,178.0,2.5,NaN,22.0,82.0,11.0,NaN,59.0,NaN,NaN,...,NaN,66.0,56.0,NaN,NaN,5.0,NaN,NaN,128.0,NaN


### 통계항목 확인
변수 이름만 봐서는 잘 와닿지 않습니다. 대체 to_ga_002가 뭘까요? 설명 파일을 봐야겠습니다. attribute 디렉토리 아래에 statistics_code.xls 파일이 있습니다. 그 파일에 설명이 있습니다. 파일을 열어보죠. 아래처럼 열어보니까 to_ga_002는 평균 가구수군요.

In [64]:
field_table = "../sgis-data/statistics_code.xls"
# 10개만 보기
pd.read_excel(field_table, parse_cols="C:D").head(10)

,통계항목,코드
0,총인구,to_in_001
1,평균나이,to_in_002
2,인구밀도,to_in_003
3,노령화지수,to_in_004
4,노년부양비,to_in_005
5,유년부양비,to_in_006
6,자료없는 집계구,to_in_999
7,총가구수,to_ga_001
8,평균가구원수,to_ga_002
9,자료없는 집계구,to_ga_999


### Geodata 불러오기 

다음은 shapefile을 불러올 차례입니다. shapefile을 geodataframe으로 바꾼 뒤 앞에서 만든 att_df와 합칠 것입니다. 여기서 주의할 점은 crs입니다. 그냥 epsg로 안되는 뭔가 customize된 걸 쓰는 거 같아요. 다행히 projection information을 제공하니 별 문제없습니다. 일단 crs 맞춰서 읽은 뒤 epsg:4326을 쓰겠습니다. 아래는 projection file의 내용을 확인하는 것입니다. 사실 datashader에서는 projected coordinates를 쓰기 때문에 꼭 crs를 변경할 필요는 없지만, geojson을 wgs84로 만들어놓으면 나중에 다른 용도로 쓰기 편하니까 일단은 wgs84로 만들겠습니다. datashader용 dataframe만들때 다시 바꾸면 되니까요.


In [65]:
%%bash
cat ../sgis-data/proj/SOP.prj

PROJCS["PCS_ITRF2000_UTM_K",GEOGCS["GCS_ITRF_2000",DATUM["D_ITRF_2000",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",1000000.0],PARAMETER["False_Northing",2000000.0],PARAMETER["Central_Meridian",127.5],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",38.0],UNIT["Meter",1.0]]

In [66]:
# 위 정보에 따른 crs string
sgis_crs = {"+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 \
            +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"}

이제 읽습니다. 제 맥북에서는 1-2분 걸립니다.

In [68]:
shapefile = os.path.join("../sgis-data/shape/", "BND_TOTAL_OA_PG.shp")
# 읽은 뒤 crs를 바꾸고 인덱스를 tot_oa_cd 컬럼으로 바꿉니다. 
raw_gdf = gpd.read_file(shapefile, crs=sgis_crs).to_crs({'init' :'epsg:4326'}).set_index("TOT_OA_CD")
raw_gdf.head()

,ADM_DR_CD,OBJECTID,SHAPE_AREA,SHAPE_LENG,geometry
TOT_OA_CD,,,,,
2401051010001,2401051,1,219266.719587,1865.991390,"POLYGON ((126.9188026753053 35.14695238753028,..."
2401051020001,2401051,2,175694.253033,1805.968661,"POLYGON ((126.9219306649181 35.14826550698864,..."
2401051030001,2401051,3,121254.392422,1948.203874,"POLYGON ((126.9212150945525 35.15165935923156,..."
2401051040001,2401051,4,53687.661571,1088.902020,"POLYGON ((126.9102208657877 35.15075423752175,..."
2401051040003,2401051,5,84698.987419,1386.972228,"POLYGON ((126.9127454550747 35.15192703550018,..."


## Join하기
앞서의 att_df와 raw_gdf를 join하고 의미없는 컬럼들을 제거할 것입니다. 그리고 NA도 그냥 0으로 채우겠습니다.


In [69]:
# join attribute table
gdf = raw_gdf.join(df)
### 999: 자료없는 집계구
drop_columns = [col for col in list(gdf) if '999' in col ] + ["OBJECTID", "SHAPE_AREA", "SHAPE_LENG"]
gdf.drop(drop_columns, inplace=True, axis=1)
# na를 0으로 채움.
gdf = gdf.fillna(0)
gdf.head()

,ADM_DR_CD,geometry,to_ga_001,to_ga_002,ga_sd_001,ga_sd_002,ga_sd_003,ga_sd_004,ga_sd_005,ga_sd_006,...,ho_pr_010,ho_pr_011,ho_pr_012,ho_gb_001,ho_gb_002,ho_gb_003,ho_gb_004,ho_gb_005,ho_gb_006,to_ho_001
TOT_OA_CD,,,,,,,,,,,,,,,,,,,,,
2401051010001,2401051,"POLYGON ((126.9188026753053 35.14695238753028,...",230.0,1.9,34.0,65.0,7.0,0.0,122.0,0.0,...,56.0,20.0,17.0,0.0,46.0,40.0,0.0,24.0,81.0,110.0
2401051020001,2401051,"POLYGON ((126.9219306649181 35.14826550698864,...",213.0,2.1,47.0,70.0,8.0,0.0,82.0,6.0,...,64.0,25.0,11.0,0.0,50.0,42.0,0.0,26.0,42.0,118.0
2401051030001,2401051,"POLYGON ((126.9212150945525 35.15165935923156,...",296.0,1.9,51.0,76.0,11.0,0.0,150.0,8.0,...,40.0,31.0,39.0,14.0,136.0,0.0,0.0,13.0,19.0,163.0
2401051040001,2401051,"POLYGON ((126.9102208657877 35.15075423752175,...",273.0,1.5,41.0,29.0,0.0,0.0,181.0,21.0,...,7.0,6.0,0.0,0.0,16.0,67.0,0.0,11.0,167.0,94.0
2401051040003,2401051,"POLYGON ((126.9127454550747 35.15192703550018,...",196.0,2.1,39.0,69.0,0.0,0.0,79.0,5.0,...,30.0,14.0,22.0,12.0,63.0,19.0,0.0,24.0,28.0,118.0


데이터프레임이 잘 결합된 것 같으니 이제 저장합시다. shapefile로 해도 좋고, postgresql로 보낼 수도 있겠습니다. 그런데 geojson이 간편하고 여러모로 범용성이 좋은 것 같으니 일단 geojson으로 저장하겠습니다. 시간이 좀 걸립니다. 5-10분 정도. 완성본 크기가 650mb 정도니까요.

In [71]:
data_dir = "../processed-data"
tract_all_geojson = os.path.join(data_dir, "tract_all.geojson")

if not os.path.exists(data_dir):
    os.mkdir(data_dir)
if os.path.exists(tract_all_geojson):
    os.remove(tract_all_geojson)

gdf.to_file(tract_all_geojson, driver='GeoJSON')

이렇게 해서 속성 데이터를 잘 결합한 geojson파일을 간단하게 만들 수 있었습니다. 잘 안되면 그냥 아래 주소에서 다운받아 processed-data에 넣으세요.
https://umich.box.com/s/og4cvpguv2rmv713wl68780tikrg4fmq

이제 이 geojson을 이용해서 dot을 만들 차례입니다.